#### 자연어 처리(NLP, Natural Language Processing)란?
- 우리의 말을 컴퓨터에게 이해시키기 위한 기술(분야)

#### 시소러스(Thesaurus)
- 유의어 사전으로, '뜻이 같은 단어(동의어)'나 '뜻이 비슷한 단어(유의어)'가 한 그룹으로 분류되어 있음
- 자연어 처리에 이용되는 시소러스는 단어 사이의 '상위와 하위' 또는 '전체와 부분'등, 더 세세한 관계까지 정의해둔 경우가 있음
- 모든 단어에 대한 유의어 집합을 만든 다음, 단어드르이 ㅗ간계를 그래프로 표현하여 단어 사이의 연결을 정의할 수 있으며, 이 '단어 네트워크'를 이용하여 컴퓨터에게 단어 사이의 관계를 가르칠 수 있음
- 하지만 사람이 수작업으로 레이블링하는 방식에는 크나큰 결점이 존재함
 * 시대 변화에 대응하기 어렵다.
 * 사람을 쓰는 비용이 크다.
 * 단어의 미묘한 차이를 표현할 수 없다.

#### 통계 기반 기법
- 말뭉치(corpus) : 대량의 텍스크 데이터로 단순 수집된 것이 아닌 자연어 처리 연구나 애플리케이션을 염두에 두고 수집된 텍스트 데이트를 뜻함
 * 말뭉치에는 문장을 쓰는 방법, 단어를 선택하는 방법, 단어의 의미 등 사람이 알고 있는 자연어에 대한 '지식'이 포함되어 있음
- 통계 기반 기법의 목표는 사람의 지식으로 가득한 말뭉치에서 자동으로, 그리고 효율적으로 그 핵심을 추출하는 것임

#### 말뭉치 전처리

In [1]:
text = 'You say goodbye and I say hello.'
text = text.lower()
text = text.replace('.', ' .')
text

'you say goodbye and i say hello .'

In [2]:
words = text.split(' ')
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [3]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
        
print(word_to_id)
print(id_to_word)
print(id_to_word[1])
print(word_to_id['hello'])

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
say
5


In [4]:
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [5]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
            
    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(word_to_id)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


#### 단어의 분산 표현(Distributional representation)
- 단어를 고정 길이의 밀집벡터(dense vector)로 표현하는 방법으로, 대부분의 원소가 0이 아닌 실수인 벡터를 뜻함

#### 분포 가설(Distributional hypothesis)
- 단어의 의미는 주변 단어에 의해 형성된다는 가설
- 즉, 단어 자체에는 의미가 없고, 그 단어가 사용된 '맥락(context)'이 의미를 형성한다는 것을 의미함
 * '맥락'이란 특정 단어를 중심에 둔 그 주변 단어를 말하는 것으로, 맥락의 크기(주변 단어를 몇개나 포함할지)를 윈도우 크기(window size)라고 함
- 분포 가설에 기초해 단어를 벡터로 나타내는 방법에는 동시발생 행렬 등이 있음
 * 동시발생 행렬(co-occurrence matrix) : 어떤 단어를 주목했을 때, 그 주변에 어떤 단어가 몇 번이나 등장하는지를 세어 집계하는 방법으로, 모든 단어에 대해 나타내줌으로써 행렬의 형태를 띄는 것

In [6]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)

print(C[0])
print(C[4])
print(C[word_to_id['goodbye']])

[0 1 0 0 0 0 0]
[0 1 0 1 0 0 0]
[0 1 0 1 0 0 0]


In [7]:
### 말뭉치가 아무리 커지더라도 자동으로 동시발생 행렬을 만들어줌
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

#### 벡터 간 유사도
- 벡터 사이의 유사도를 측정하는 방법에는 벡터의 내적이나 유클리드거리 등을 꼽을 수 있지만 코사인 유사도를 자주 이용함
 * 코사인 유사도(cosine similarity) : 두 벡터가 가리키는 방향이 얼마나 비슷한가를 나타내는 척도로, 두 벡터의 방향이 완전히 같다면 1, 반대라면 -1이 됨

In [8]:
### 해당 함수는 코사인 유사도를 구할 수 있지만 인수로 제로 벡터(원소가 0인 벡터)가 들어오면 'divide by zero' 오류가 발생함
def cos_similarity(x, y):
    nx = x / np.sqrt(np.sum(x**2))   ### x의 정규화
    ny = y / np.sqrt(np.sum(y**2))   ### y의 정규화
    return np.dot(nx, ny)

### 제로 벡터의 문제를 해결하기 위해 최종 계산 결과에는 영향을 주지 않는 eps(엡실론)을 더해주는 방법을 사용할 수 있음
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

In [9]:
# import sys
# sys.path.append('..')
# from common.util import preprocess, create_co_matrix, cos_similarity

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]   ### 'you'의 단어 벡터
c1 = C[word_to_id['i']]   ### 'i'의 단어 벡터
print(cos_similarity(c0, c1))

0.7071067758832467


- 어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 것도 가능
 > query : 검색어(단어)
 > word_to_id : 단어에서 단어 ID로의 딕셔너리
 > id_to_word : 단어 ID에서 단어로의 딕셔너리
 > word_matrix : 단어 벡터들을 한데 모은 행렬, 각 행에는 대응하는 단어의 벡터가 저장되어 있다고 가정한다.
 > top : 상위 몇 개까지 출력할지 설정

In [10]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    ### 검색어를 꺼낸다.
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return
    
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    ### 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    ### 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return
        

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067758832467
 i: 0.7071067758832467
 hello: 0.7071067758832467
 say: 0.0
 and: 0.0


In [11]:
### argsort()는 배열 인덱스의 정렬을 바꾸는데 사용하는 매서드로, 넘파이 배열의 원소를 오름차순으로 정렬함(반환값은 인덱스)
x = np.array([100, -20, 2])
print(x.argsort())
print((-x).argsort())

[1 2 0]
[0 2 1]


#### 상호정보량
- 동시발생 행렬의 원소는 두 단어가 동시에 발생한 횟수를 나타내는 것으로, 발생 횟수라는 특징은 고빈도 단어(많이 출현하는 단어)를 위주이기 떄문에 좋은 특징이라 할 수 없음
- 점별 상호정보량(PMI, pointwise mutual information)
 * 동시발생 횟수 관점에서의 고빈도 단어 문제를 개선하기 위한 척도로, PMI값이 높을수록 관련성이 높음
 * PMI는 확률 변수 x와 y에 대해 다음 식으로 정의됨
  > PMI(x,y) = log2(P(x,y)/(P(x)P(y)))
  > - P(x) : x가 일어날 확률
  > - P(y) : y가 일어날 확률
  > - P(x,y) : x와 y가 동시에 일어날 확률
 * 동시발생 행렬에서의 PMI는 다음과 같음
  > PMI(x,y) = log2((C(x,y)*N)/(C(x)C(y)))
  > - C : 동시발생 행렬
  > - C(x,y) : 단어 x와 y가 동시발생하는 횟수
  > - C(x), C(y) : 각각 단어 x와 y의 등장 횟수
  > - N : 말뭉치에 포함된 단어의 수
- 양의 상호정보량(PPMI, positive PMI)
 * 두 단어의 동시발생 횟수가 0이면 발생하는 문제를 피하기 위해 실제로 구현할 때 사용되는 척도
  > PPMI(x,y) = max(0, PMI(x,y))
 * 해당 방법은 PMI가 음수일 때 0으로 취급하여 단어 사이의 관련성을 0 이상의 실수로 나타냄

In [12]:
### verbose : 진행상황 출력 여부
def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C,  dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / S[j]*S[i] + eps)
            M[i, j] = max(0, pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total//100)  == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3)   ### 유효 자릿수를 세 자리로 표시
print('동시발생 행렬')
print(C)
print('-'*50)
print('PPMI')
print(W)

동시발생 행렬
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
--------------------------------------------------
PPMI
[[0.    1.807 0.    0.    0.    0.    0.   ]
 [5.807 0.    4.807 0.    4.807 4.807 0.   ]
 [0.    2.807 0.    3.807 0.    0.    0.   ]
 [0.    0.    3.807 0.    3.807 0.    0.   ]
 [0.    2.807 0.    3.807 0.    0.    0.   ]
 [0.    2.807 0.    0.    0.    0.    4.807]
 [0.    0.    0.    0.    0.    2.807 0.   ]]


- PPMI 행렬도 말뭉치의 어휘 수가 증가함에 따라 각 단어 벡터의 차원 수도 증가한다는 문제가 존재함
- 또한, 행렬의 원소 대부분이 0이라는 것은 각 원소의 '중요도'가 낮다는 의미로, 이런 벡터는 노이즈에 약하고 견고하지 못하는 약점이 있음

#### 차원 감소(Dimensionality reduction)
- '중요한 정보'는 최대한 유지하면서 벡터의 차원을 줄이는 방법을 뜻함
- 차원 감소의 핵심은 희소벡터에서 중요한 축을 찾아내어 더 적은 차원으로 다시 표현하는 것으로, 희소벡터를 밀집벡터로 변환시킨다고 생각하면 됨
 * 희소벡터(sparse vector) : 원소 대부분이 0인 행렬 또는 벡터
- 특이값분해(SVD, singular value decomposition)
 * 임의의 행렬을 세 행렬의 곱으로 분해
  > X = US(V**T)
  > - U와 V는 직교행렬(orthogonal matrix)이고, 그 열벡터는 서로 직교함
  > - S는 대각행렬(diagonal matrix)로 대각성분 외에는 모두 0인 행렬임

In [13]:
### linalg는 선형대수(linear algebra)의 약자
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)
U, S, V = np.linalg.svd(W)
print(C[0])
print(W[0])
print(U[0])
print(U[0, :2])

[0 1 0 0 0 0 0]
[0.    1.807 0.    0.    0.    0.    0.   ]
[ 0.000e+00  1.748e-01  3.821e-02 -1.110e-16 -1.110e-16 -9.839e-01
 -1.272e-16]
[0.    0.175]


In [14]:
import matplotlib.pyplot as plt
for word, word_id in word_to_id.items():
    plt.annotate(word, (U[word_id, 0], U[word_id, 1]))
    
plt.scatter(U[:, 0], U[:, 1], alpha=0.5)
plt.show()

<Figure size 640x480 with 1 Axes>

#### Conclusion
- 통계 기반 기법은
 1. 말뭉치로부터 단어의 의미를 자동으로 추출하고,
 2. 그 의미를 벡터로 표현하고,
 3. 단어의 동시발생 행렬을 만들고,
 4. PPMI(양의 상호정보량) 행렬로 변환한 다음,
 5. 안전성을 높이기 위해 SVD(특이값분해)를 이용해 차원을 감소시켜 각 단어의 분산 표현을 만들어냄